## SET UP

In [ ]:
## cmd -> cd C:\Users\infomax\anaconda3\Scripts  

In [1]:
pip install --upgrade pip3
pip install calcbsimpvol

from calcbsimpvol import calcbsimpvol
#시스템 설정 가서 path에 anaconda3>lib 추가해준 후, reboot 하고 나서 import 실행.

pip install --upgrade scipy
pip install --upgrade pandas
pip install pandas_datareader
pip install --upgrade numpy

In [4]:
import numpy as np 
from math import log, e
from scipy.stats import norm

In [33]:
class BSM:
    def __init__ (self, args):
        self.mkt_prc = float(args[0])
        self.Type = int(args[1])
        self.S = float(args[2])
        self.K = float(args[3])
        self.r = float(args[4])
        self.q = float(args[5])
        self.T = float(args[6])/365.0

        self.sigma = self.imp_vol()
        self.sigmaT = self.sigma * self.T **0.5

        self.d1 = (log(self.S / self.K)+(self.r - self.q + 0.5*(self.sigma**2))*self.T) / self.sigmaT
        self.d2 = self.d1 - self.sigmaT    


    def imp_vol(self): 
        cp = np.asarray(self.Type)
        P = np.asarray([self.mkt_prc])
        K = np.asarray([self.K])
        S = np.asarray([self.S])
        tau = np.asarray([self.T])
        r = np.asarray(self.r)
        q = np.asarray(self.q)

        imvol = calcbsimpvol(dict(cp=cp, P=P, S=S, K=K, tau=tau, r=r, q=q))
        imvol = np.ndarray.item(imvol)
        return imvol


    def delta(self) :
        dfq = e**(-self.q * self.T)
        if self.Type == 1:      #Option Type is arranged.
            return dfq * norm.cdf(self.d1)
        else:
            return dfq * (norm.cdf(self.d1) -1)


    def gamma(self):
        return e** (-self.q * self.T) * norm.pdf(self.d1) * (self.S*self.sigmaT)


    def vega(self):
        return 0.01 * self.S * e **(-self.q * self.T) * norm.pdf(self.d1) * self.T **0.5


    def theta(self):
        df = e ** -(self.r * self.T)
        dfq = e ** (-self.q * self.T)
        tmptheta = (1.0 / 365.0) * (-0.5 * self.S * dfq * norm.pdf(self.d1) * self.sigma / (self.T **0.5) * self.Type * (self.q * self.S * dfq * norm.cdf(self.Type * self.d1) - self.r *self.K * df * norm.cdf(self.Type *self.d2)))
        return tmptheta


    def rho(self):
        df = e** -(self.r *self.T)
        return self.Type * self.K * self.T *df *0.01 *norm.cdf(self.Type * self.d2)

### 클래스 호출과 동시에 실행시키는 init.
### 클래스 내의 모든 함수의 첫번째 인자(argument)가 self임.
### 변수앞에 self.를 붙이면, 클래스와 함수에서 사용가능한 변수가 된다. 
### - instant variable.
### 이렇게 하면 그릭스 계산시 인자를 따로 받을 필요가 없음. 


In [34]:
mkt_prc = 0.31
op_Type = 1   #1은 뭐고, 1이 아닌건 뭐고? 누가 정했나?
S = 268.55
K = 275.0
q= 0.0
r = 0.01
T = 9

args = [mkt_prc, op_Type, S, K, r, q, T]
BSM = BSM(args)

In [36]:
imp_vol = BSM.imp_vol()
delta = BSM.delta()
gamma = BSM.gamma()
vega = BSM.vega()
theta = BSM.theta()
rho = BSM.rho()

In [38]:
print('Imp_Vol = ', round(imp_vol, 4))
print('Delta = ', round(delta, 4))
print('Gamma = ', round(gamma, 4))
print('Vega = ', round(vega, 4))
print('Theta = ', round(theta, 4))
print('Rho = ', round(rho, 4))

Imp_Vol =  0.126
Delta =  0.1196
Gamma =  1.0609
Vega =  0.0842
Theta =  0.0188
Rho =  0.0078


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: cycler>=0.10 in c:\users\infomax\anaconda3\lib\site-packages (from matplotlib->calcbsimpvol) (0.10.0)



DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip
